<h1 style="color: blue; text-align: center;">Multi-task Cascaded Convolutional Networks (MTCNN) for Face Detection and Facial Landmark Alignment Using Parallel</h1>

---
## Members of Team
| MSSV | Full Name | ID Github |
| ---- | ---- | -------- |
| 1612835 | Bùi Trọng Xuyến | btxuyenhcmus |
| 1612165 | Nguyễn Đào Vinh Hải | NDVHaiHCMUS |
| 1612859 | Nguyễn Đình Hữu | NguyenDinhHuu-HCMUS |

---
### 1.  Mô tả ứng dụng

---
### 2. Cài đặt tuần tự

---
### 3. Cài đặt song song (trên GPU)

---
### 4. Cài đặt song song (trên GPU) + tối ưu hóa

---
### 5. Nhìn lại quá trình làm đồ án

---
### 6. Tài liệu tham khảo